In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from selenium import webdriver
import time
import locale

In [2]:
MIN_PVP = 0.75
MAX_PVP = 2
MIN_STOCK = 5
MAX_STOCK = 21
MIN_DY = 0.65
MIN_MARKET_VALUE = 5000000

In [3]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [4]:
PATH_LINK_FII_NAMES = "https://www.fundsexplorer.com.br/ranking"

In [5]:
request_headers = {'User-Agent': 'Chrome/126.0.0.0' }

In [6]:
driver = webdriver.Chrome()
url = 'https://www.fundsexplorer.com.br/ranking'

In [7]:
def take_html(url):
  driver.get(url)
  time.sleep(5)
  html = driver.page_source
  return BeautifulSoup(html, 'html.parser')

html = take_html(PATH_LINK_FII_NAMES)
driver.quit()

In [8]:
infos_not_formated = {
"post_title": [],
"setor": [],
"valor": [],
"liquidezmediadiaria": [],
"pvp": [],
"media_yield_12m": [],
"patrimonio": [],
"volatility": [],
"numero_cotista": [],
}

In [9]:
def take_infos(html):
  table = html.find(class_="default-fiis-table__container__table__body").find_all("tr")

  for row in table:
    row = row.find_all("td")
    for td in row:
      data_column = td["data-collum"].replace("collum-", "") 
      if data_column in infos_not_formated:
          infos_not_formated[data_column].append(td.text) 

take_infos(html)

In [10]:
columns_formated = {
  "post_title": "Código",
  "setor": "Segmento",
  "valor": "Cotação",
  "liquidezmediadiaria": "Liquidez Diaria",
  "pvp": "P/VP",
  "media_yield_12m": "Média DY (12M)",
  "patrimonio": "Patrimônio",
  "volatility": "Volatilidade",
  "numero_cotista": "Número de cotistas",
}

In [11]:
def format_columns():
  infos = {}
  for column in columns_formated:
    infos[columns_formated[column]] = infos_not_formated[column]
    
  return infos

infos_not_formated = format_columns()

In [12]:
def format_values():
  infos = {}
  for column in infos_not_formated:
    for data in infos_not_formated[column]:
      if column not in infos:
        infos[column] = []
      
      infos[column].append(str(data).replace(".", "").replace(",", ".").replace("%", ""))
  
  return infos

infos = format_values()

In [13]:
infos["Média dividendos"] = []
infos["Bola de neve"] = []

for i in range(len(infos["Código"])):
  dy_mean = infos["Média DY (12M)"][i]
  dy_mean = float(dy_mean if dy_mean != "N/A" else 0)
  stock = infos["Cotação"][i]
  stock = float(stock if stock != "N/A" else 0)
  
  try:
    infos["Média dividendos"].append(stock * (dy_mean / 100))
  except:
    infos["Média dividendos"].append(0)
  
  magic_number = int(1 / (dy_mean / 100)) if dy_mean != 0 else "None"
  
  infos["Bola de neve"].append(magic_number) 

In [14]:
infos_list = []

for i in range(len(infos["Código"])):
  info = []
  for column in infos:
    info.append(infos[column][i])
  
  infos_list.append(info)

In [15]:
df = pd.DataFrame(infos_list, columns=list(infos.keys()))
df = df.set_index("Código")

In [16]:
df

,Segmento,Cotação,Liquidez Diaria,P/VP,Média DY (12M),Patrimônio,Volatilidade,Número de cotistas,Média dividendos,Bola de neve
Código,,,,,,,,,,
AAGR11,Indefinido,99.80,15911.67,N/A,1.49,N/A,60.35,0,1.487020,67
AAZQ11,Indefinido,7.28,1537150.04,0.77,1.30,226760194.13,17.28,28223,0.094640,76
ABCP11,Shoppings,68.14,60509.60,0.67,0.76,477813607.64,15.01,16357,0.517864,131
AFHI11,Papéis,97.43,1841712.32,1.01,1.03,383985644.70,6.95,38421,1.003529,97
AGRX11,Outros,9.76,296059.83,N/A,1.30,N/A,16.03,0,0.126880,76
...,...,...,...,...,...,...,...,...,...,...
YUFI11B,Indefinido,N/A,N/A,N/A,0.00,N/A,N/A,0,0.000000,None
ZAGH11,Serviços Financeiros Diversos,12.00,1352.86,0.12,0.00,46671844.10,488.88,120,0.000000,None
ZAVC11,Indefinido,9.89,N/A,0.99,0.00,30835569.93,N/A,84,0.000000,None


In [17]:
df.info()
df = df[df["P/VP"] != "N/A"]
df = df[df["Cotação"] != "N/A"]
df = df[df["Liquidez Diaria"] != "N/A"]
df = df[df["Média DY (12M)"] != "N/A"]
df = df[df["Patrimônio"] != "N/A"]
df = df[df["Volatilidade"] != "N/A"]
df = df[df["Número de cotistas"].astype(int) > 100]
df = df[df["Bola de neve"] != "None"]

df_data_science = df.copy() 

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, AAGR11 to ZIFI11
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Segmento            478 non-null    object 
 1   Cotação             478 non-null    object 
 2   Liquidez Diaria     478 non-null    object 
 3   P/VP                478 non-null    object 
 4   Média DY (12M)      478 non-null    object 
 5   Patrimônio          478 non-null    object 
 6   Volatilidade        478 non-null    object 
 7   Número de cotistas  478 non-null    object 
 8   Média dividendos    478 non-null    float64
 9   Bola de neve        478 non-null    object 
dtypes: float64(1), object(9)
memory usage: 41.1+ KB


In [18]:
df["Cotação"] = df["Cotação"].astype(float)
df["Liquidez Diaria"] = df["Liquidez Diaria"].astype(float)
df["P/VP"] = df["P/VP"].astype(float)
df["Média DY (12M)"] = df["Média DY (12M)"].astype(float)
df["Patrimônio"] = df["Patrimônio"].astype(float)
df["Volatilidade"] = df["Volatilidade"].astype(float)
df["Número de cotistas"] = df["Número de cotistas"].astype(int)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, AAZQ11 to ZAVI11
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Segmento            324 non-null    object 
 1   Cotação             324 non-null    float64
 2   Liquidez Diaria     324 non-null    float64
 3   P/VP                324 non-null    float64
 4   Média DY (12M)      324 non-null    float64
 5   Patrimônio          324 non-null    float64
 6   Volatilidade        324 non-null    float64
 7   Número de cotistas  324 non-null    int32  
 8   Média dividendos    324 non-null    float64
 9   Bola de neve        324 non-null    object 
dtypes: float64(7), int32(1), object(2)
memory usage: 26.6+ KB


In [19]:
df = df[df["P/VP"] > MIN_PVP]
df = df[df["P/VP"] < MAX_PVP]
df = df[df["Cotação"] > MIN_STOCK]
df = df[df["Cotação"] < MAX_STOCK]
df = df[df["Média DY (12M)"] > MIN_DY]
df = df[df["Patrimônio"] > MIN_MARKET_VALUE]
df.head()

,Segmento,Cotação,Liquidez Diaria,P/VP,Média DY (12M),Patrimônio,Volatilidade,Número de cotistas,Média dividendos,Bola de neve
Código,,,,,,,,,,
AAZQ11,Indefinido,7.28,1537150.04,0.77,1.30,2.267602e+08,17.28,28223,0.094640,76
ALZM11,Fundo de Fundos,8.02,168211.21,0.96,0.99,1.274195e+08,15.12,10317,0.079398,101
APTO11,Imóveis Residenciais,9.54,20441.20,0.94,0.94,4.661087e+07,19.60,9871,0.089676,106
ARRI11,Papéis,9.04,442039.29,1.03,1.10,1.346838e+08,9.47,26524,0.099440,90
BBIG11,Serviços Financeiros Diversos,9.25,630726.46,0.95,1.67,9.629636e+08,14.07,15464,0.154475,59


In [20]:
df_locale = df.copy()

def currency_format(value):
  return locale.currency(value)

def decimal_format(value):
  return str(value).replace(".", ",")

def percentage_format(value):
  return decimal_format(value) + "%"

def market_value_format(value):
  result = value
  if value > 1_000_000_000:
    result = str(int(value / 1_000_000_000)) + " B"
  elif value > 1_000_000:
    result = str(int(value / 1_000_000)) + " M"
  elif value > 1_000:
    result = str(int(value / 1_000)) + " mil"
  
  return f"R$ {result}"

df_locale["Cotação"] = df_locale["Cotação"].apply(currency_format) 
df_locale["Liquidez Diaria"] = df_locale["Liquidez Diaria"].apply(currency_format) 
df_locale["P/VP"] = df_locale["P/VP"].apply(decimal_format) 
df_locale["Média DY (12M)"] = df_locale["Média DY (12M)"].apply(percentage_format)
df_locale["Patrimônio"] = df_locale["Patrimônio"].apply(market_value_format) 
df_locale["Volatilidade"] = df_locale["Volatilidade"].apply(percentage_format)
df_locale["Média dividendos"] = df_locale["Média dividendos"].apply(currency_format)

df_locale.head() 


,Segmento,Cotação,Liquidez Diaria,P/VP,Média DY (12M),Patrimônio,Volatilidade,Número de cotistas,Média dividendos,Bola de neve
Código,,,,,,,,,,
AAZQ11,Indefinido,"R$ 7,28","R$ 1537150,04","0,77","1,3%",R$ 226 M,"17,28%",28223,"R$ 0,09",76
ALZM11,Fundo de Fundos,"R$ 8,02","R$ 168211,21","0,96","0,99%",R$ 127 M,"15,12%",10317,"R$ 0,08",101
APTO11,Imóveis Residenciais,"R$ 9,54","R$ 20441,20","0,94","0,94%",R$ 46 M,"19,6%",9871,"R$ 0,09",106
ARRI11,Papéis,"R$ 9,04","R$ 442039,29","1,03","1,1%",R$ 134 M,"9,47%",26524,"R$ 0,10",90
BBIG11,Serviços Financeiros Diversos,"R$ 9,25","R$ 630726,46","0,95","1,67%",R$ 962 M,"14,07%",15464,"R$ 0,15",59


In [21]:
df_locale.to_csv(f"result-{datetime.now().strftime("%d-%m-%Y")}.csv", sep=";")